# Algotrading - Aula 4

Montando um Backtesting simples
___

Vamos começar importando algumas bibliotecas úteis

In [ ]:
# Pequeno ajuste inicial

!pip install "numpy<1.25" --upgrade

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from IPython import display

Instalar também uma biblioteca também útil para pegar dados de fechamento direito do yahoo finance:

In [ ]:
!pip install yfinance

Agora vamos baixar dados de fechamento (**Historical Data**) de 5 anos diretamente do site

In [ ]:
import yfinance as yf
from datetime import datetime, timedelta

ticker = '^BVSP'

# Selecionado uma data de hoje menos 5 anos
start_date = (datetime.now() - timedelta(days=5*365))

# Formatando para Ymd
start_date = start_date.strftime('%Y-%m-%d')

# Selecionando a data de ontem
end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

data = yf.download(ticker, start=start_date, end=end_date, interval='1d', auto_adjust=True)

Agora que sabemos como separar o que interessa, vamos começar a montar nosso dataframe

In [ ]:
# Remove o ticker do index
ibov = data.droplevel(1, axis=1)
ibov

Voltando aos dados diários, vamos dar uma olhadinha brevemente nos retornos

In [ ]:
ibov_ret = ibov["Close"].pct_change().dropna()*100
ibov_ret.head(10)

Explorando os dados

In [ ]:
# Medidas Resumo
ibov_ret.describe()

In [ ]:
# Histograma
ibov_ret.plot.hist(density=True);

In [ ]:
# Histograma
import scipy.stats as stats
stats.probplot(ibov_ret, dist="norm", plot=plt);

Obs: Deveríamos fazer o teste de normalidade dos dados para ter certeza!

In [ ]:
# Boxplot
ibov_ret.plot(kind='box');

Calculando o log-retorno

In [ ]:
ibov_logr = ibov.Close.apply(np.log).diff().dropna()*100
ibov_logr.head()

Apenas verificando se log-retorno e retorno são muito diferentes para esses dados

In [ ]:
(ibov_ret - ibov_logr).plot();

### Simulando uma Estratégia de Trading

Vamos começar olhando como interagir com os dados:

In [ ]:
# Percorrendo todos os preços de fechamento
for price in ibov.Close:
    print(price)

Imagine agora que você comprou 1 contrato do instrumento sem alavancagem no próprio preço no primeiro ponto:

In [ ]:
# Primeiro ponto:
print(ibov.Close.iloc[0])

Obs: Não é possível comprar IBOV diretamente como uma ação. O índice bovespa é formado por uma cesta de ações ponderadas pelo seu volume histórico. Mas é possível comprar ETF (Exchange Traded Fund) ou Futuro (derivativo) que segue o índice. https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/ibovespa.htm

In [ ]:
# Montando a alocação

k = 1_000_000
q = k // ibov.Close.iloc[0]
k -= q * ibov.Close.iloc[0]

notional = q * ibov.Close.iloc[0]
print(f"Quantidade de instrumentos comprados: {q}")
print(f"Preço de compra: {ibov.Close.iloc[0]}")
print(f"Tamanho da posição: {notional}")
print(f"Caixa restante: {k}")

*Notional* é uma denominação para a posição em termos de dinheiro, usado principalmente para posições alavancadas de derivativos. Ela se contrapõe à palavra *position*, que pode denominar tanto a quantidade de dinheiro ou a quantidade de ações/contratos.

---

Agora imagine que vendeu no último dia:

In [ ]:
# Resultado em termos monetários
result = q * ibov.Close.iloc[-1] - notional
print(result)

Esse é o retorno financeiro (em dinheiro) que você obteve **comprando** no primeiro dia e **vendendo** no último dia da simulação. Essa estratégia é chamada de **BUY & HOLD** ou **BnH**.

E qual foi o retorno percentual?

In [ ]:
# Em percentual (retorno):
print(f'{result/notional*100:0.2f}%')

Resultado: Se comprou IBOV 5 anos atrás e vendeu ontem, ganhou cerca de R$ 356 mil investindo cerca de R$ 917 mil.

PnL (Profit & Loss): 36.93\%

Parece um bom negócio? Será que é tão simples?

___

Para contar **toda** a história do **trade**, queremos saber quanto ela rendeu ao longo do tempo usando marcação a mercado (Mark to Market - MtM).

MtM é uma forma de saber o seu patrimônio atual, ou seja, o seu dinheiro em caixa somado ao valor atual dos seus ativos. É uma forma de gerenciar melhor os riscos, sem cair na falácia do "Não vendi, não perdi", uma forma de auto-enganação.

---

Precisamos analisar o quanto seria o meu resultado para cada dia que passa durante o trade:

In [ ]:
# Comprei no primeiro ponto
k = 1_000_000
q = k // ibov.Close.iloc[0]
k -= q * ibov.Close.iloc[0]

notional = q * ibov.Close.iloc[0]

for price in ibov.Close:
    # Para cada ponto, calculo meu resultado se fosse vender
    print(f"Retorno acumualdo diário: {q * price - notional}")

Não foi muito útil, vamos gerar o gráfico:

In [ ]:
# Comprei no primeiro ponto
k = 1_000_000
q = k // ibov.Close.iloc[0]
k -= q * ibov.Close.iloc[0]

notional = q * ibov.Close.iloc[0]

result = []
for price in ibov.Close:
    # Para cada ponto, calculo meu resultado se fosse vender e guardo em uma lista
    result.append(q * price - notional)

pd.Series(result).plot();

Agora sim, e não pareceu tão bonito quanto o resultado final. Qual foi o pior momento?

In [ ]:
min(result)

In [ ]:
print(f'{min(result)/notional*100:0.2f}%')

Esse resultado é o **Max Drawdown** (**MDD**), ou seja, o maior prejuízo pontual ao longo do tempo. Você consegue calcular o Max Profit?

Outra medida a ser observada é a taxa de acerto (**Hitting Ratio** ou **HR**), mas como só foi feita 1 operação, essa medida não faz muito sentido ainda.

E a volatilidade do resultado?

In [ ]:
pd.Series(result).describe()

In [ ]:
(pd.Series(result[1:])*100/notional).describe()

Para calcular a volatilidade anualizada, precisa anualizar o valor do desvio padrão (ver os slides da aula).
___
 
Também é possível fazer o chamado **backtesting vetorial**:

In [ ]:
ibov_vet = ibov.copy()
ibov_vet['signal'] = [1] + [0]*(len(ibov_vet)-2) + [-1]

ibov_vet


In [ ]:
ibov_vet['flow'] = - ibov_vet['signal'] * ibov_vet['Close']

ibov_vet

In [ ]:
ibov_vet['flow'].sum()

Esse é o resultado por instrumento. Para recuperar o número obtido anteriormente, multiplique pela quantidade comprada (nove) e verifique o batimento do resultado.

É possível reproduzir ainda algumas métricas, mas esse tipo de backtesting é muito limitado.

---

Para termos uma maior acurácia no backtesting, vamos fazer **evento a evento**, o que simularia realmente o cotidiano da vida real.

Alguns detalhes:

Sempre trabalhar com fluxo de caixa:
 * Toda vez que compra, dinheiro sai do bolso
 * Toda vez que vende, dinheiro entra

Sempre usar o MtM para estimar o resultado a cada evento (no caso dia)

---

Em python também é possível montar um gráfico mais interativo:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,5));

cash = 1_000_000 # indica quanto dinheiro tem no bolso
position = 0 # indica a posição atual. 0 é zerado
result = [] # lista para montar o gráfico

# Vamos simular apenas os 100 primeiros pontos (dias)
for price in ibov.Close.iloc[:180]:
    
    # Se a posição está zerada, compra
    if position == 0:
        q = cash // price # Quantidade que pode comprar
        cash -= q * price # Fluxo de caixa negativo
        position = 1 # Indica que está comprado em 1

    # Qual o meu resultado atual
    result.append(cash + q * price * position) # Mark To Market!
   
    # Fazendo o plot a cada iteração:
    ax.plot(result, color='blue')    
    display.clear_output(wait=True)
    display.display(fig)
    
ax.plot(result, color='blue');
display.clear_output(wait=True)

---

#### NOVA ESTRATÉGIA

Vamos simular um clássico: 2-period RSI, apresentado por Larry Connors no livro Short-Term Trading Strategies That Work (2008).

A ideia é utilizar um indicador técnico chamado Relative Strenght Index (RSI) com o período curtíssimo de valor 2 (normalmente usa-se 14 observações).

RSI é um índice normalizado entre 0 e 100, utilizando uma razão entre média de subidas e quedas, suavizadas por uma média móvel exponencial. Mais detalhes mais tarde no curso.

* Entrada: no livro, Connors sugere a compra quando o valor do RSI(2) for menor ou igual a 10
* Saída: quando o RSI(2) atingir o valor maior ou igual a 80

Vamos montar um backtesting dessa estratégia!

Primeiro, vamos instalar uma biblioteca de auxílio:

In [ ]:
!pip install pandas-ta

Calculando o RSI(2):

In [ ]:
import pandas_ta as ta
ibov['rsi'] = ibov.ta.rsi(length=2)
ibov = ibov.dropna()
ibov

In [ ]:
ibov['rsi'].describe()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10,5));

ibov['Close'].iloc[-150:].plot(ax=ax[0]);
ibov['rsi'].iloc[-150:].plot(ax=ax[1]);

Cálculo rudimentar do PnL:

In [ ]:
# Protótipo da estratégia:
k = 1_000_000
position = 0
PV = [] # Patrimonio líquido

for row in ibov.itertuples():
    if row.rsi < 10 and position == 0:
        position = k // row.Close
        k -= position * row.Close
    elif row.rsi > 80 and position != 0:
        k += position * row.Close
        position = 0
    
    PV.append(k + position * row.Close)

pd.Series(PV).plot();

In [ ]:
print(f"PnL acumulado: {PV[-1] - 1_000_000}")

Vocês acham?

Podemos melhorar esse número?

Só o PnL é suficiente para avaliar a estratégia?

Mais importante: está certo isso? (dica: falta carrego e custos)

___

### Lista: Exercício 1 - 17/Set até 14h00

* Implementar as métricas dos slides da aula. Considerar apenas: Profitability, Risk e Performance Metrics
* Implementar o carrego (rendimento do dinheiro em caixa). Considerar CDI hipotético de 10% ao ano
* Desconsiderar os custos por enquanto. Assumir que entra no preço de fechamento do dia usado no RSI
* Entregar um **IPYNB (Jupyter Notebook em PYTHON)** com o código e o gráfico da simulação do resultado
* Prazo: **17/Set até 14h00** via Blackboard (Após esse prazo será considerado atrasado)
* Entrega obrigatória! Entregas incompletas ou que não cumprem os requisitos acima, não serão consideradas e precisarão ser reentregues com atraso.
* Estritamente individual